# A Trainable Simulator: using unsupervised learning in conjunction with computational methods to rewrite our equations, applied to multiphase flow

## Alejandro Francisco Queiruga
### Lawrence Berkeley National Lab

## Don't think neural networks, think *differentiable programming*

## How do we improve what we have now?

1. What problems am I trying to solve?
2. Why are our existing multiphase simulators so ugly?
3. Rephrase our equations to not have primary variables
4. The classical pendulum as an analogy
5. Learning primary variables with autoencoders
6. Unsupervised learning for phases
7. Testing it out

## What kinds of problems?

<img src='slides/simple_wells.png' width=500>

Single phase Darcy's law is easy:

\begin{equation}
\frac{\partial p}{\partial t} = \frac{1}{M} \nabla \cdot \frac{k}{\mu} \nabla p
\end{equation}

## What kinds of problems?

<img src='slides/complex_wells.png' width=500>



- Multicomponent mass and heat transfer
- Big ranges in our problems: depths, temperature fluxes, chemical
  compositions, percipates...
- Phase changes!


## Just multiphase representation is the hard part

<img src='slides/cstr.png' width=500>

- Transition between phases
- Equilibria with coexisting phases

Community focus is on turbulence in navier stokes, but that's pure liquid.

**_Our goal:_ Write a better true-physics multiphase flow solver.**

## What is a phase?

![watereos](figures/phase_diagram.png)

<center>Phase diagram of water</center>

- Sudden changes in material properties
- Human object recognition distinguishes phases

> Phase transitions occur when the thermodynamic free energy of a system is non-analytic for some choice of thermodynamic variables (cf. phases).

https://en.wikipedia.org/wiki/Phase_transition

## Water EOS

In [3]:
from IPython.display import IFrame
IFrame('figures/water_eos.html',width=700,height=500)

Used IAPWS empirical fits to make this data set. Colored by phase, including mixtures.

## Water EOS

<img src="figures/water_eos.png" width=500>
Used IAPWS empirical fits to make this data set. Colored by phase, including mixtures.

# Curve fits are complicated (and arbitrary):

```python
def gibbs_liquid_h2o(T,p): # From IAPWS '97
    p1_star = 1.653e7
    T1_star  = 1.386e3
    n1 = [ 0.14632971213167e00, -0.84548187169114e00,
          -3.7563603672040e+00,  3.3855169168385e+00, 
          -0.95791963387872e00,  0.15772038513228e00,
          -1.6616417199501e-02,  8.1214629983568e-04, 
           2.8319080123804e-04, -6.0706301565874e-04,
          -1.8990068218419e-02, -3.2529748770505e-02, 
          -2.1841717175414e-02, -5.2838357969930e-05,
          -4.7184321073267e-04, -3.0001780793026e-04, 
           4.7661393906987e-05, -4.4141845330846e-06,
          -7.2694996297594e-16, -3.1679644845054e-05, 
          -2.8270797985312e-06, -8.5205128120103e-10,
          -2.2425281908000e-06, -6.5171222895601e-07, 
          -1.4341729937924e-13, -4.0516996860117e-07,
          -1.2734301741641e-09, -1.7424871230634e-10, 
          -6.8762131295531e-19,  1.4478307828521e-20,
           2.6335781662795e-23, -1.1947622640071e-23, 
           1.8228094581404e-24, -9.3537087292458e-26  ]
    i1 = [  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,   
            1,  2,  2,  2,  2,  2,  3,  3,  3,  4,  4,  4,  5,    
            8,  8, 21, 23, 29, 30, 31, 32  ]
    j1 = [ -2, -1,   0,   1,   2,   3,   4,   5, -9, -7, -1,  0,  1,    
               3, -3,   0,   1,   3,  17,  -4,   0,  6, -5, -2, 10, -8,   
              -11, -6, -29, -31, -38, -39, -40, -41  ]
    p_i = p/p1_star
    t_i = T1_star/T
    return R*T*sum([ n*(7.1-p_i)**I*(t_i-1.222)**J
            for n,I,J in zip(n1,i1,j1)])
density_region1,enthalpy_region1 = density_enthalpy(gibbs_region1)
```

## How do we solve this?

- Different equations and **primary variables** for each phase, plus a state tag.
- Transition with a **finite state machine**

1. Gas: $X = \{p,T\}$  
  $\rho_{gas}(p,T) = $ one fit
2. Liquid: $X = \{p,T\}$  
  $\rho_{liq}(p,T) = $ another fit
3. Liq-Gas: $X = \{S_{gas},T\}$  
  $\rho_{mix}(S,T) = S \rho_{gas}(p^*(T),T)+ (1-S) \rho_{liq}(p^*(T),T)$ 


- Lots of bug ridden coding!
- Need numerical differentiation!
- Slow convergence!
- Easily unstable! Need a lot of hacks, like overshoots:  
$p_{new} = (1+10^{-6})p_{old}$

## State Machines Get Very Complex:

<div style="float:left;">
<h4> Water </h4>
<img src='slides/phase_4.png' width=500>
</div>
<div style="float:left;">
    <h4> Methane Hydrate</h4>
<img src='figures/hydrate_statemachine.png' width=300></div>

## How do we solve this better?

Take a step back:
- Forget about phases and states and the equations we wrote.
- **It's just data**: We have observations and expect to stay on these observations.
- We just need _**any**_ representation of this constraint.

## A Constrained Differential Algebraic Equation


Solve for $\rho(t), \, h(t), \, p(t),\, \text{and}\, T(t)$ satisfying:
\begin{align}
\partial_t \rho & = \nabla \cdot \mathbf{k}\nabla p + r & \quad \}\text{mass balance}\\
\partial_t (\rho h-p) & = \nabla \cdot \mathbf{k'}\nabla T + s & \quad \}\text{energy balance}\\
\end{align}
such that they lie on the material EOS,
\begin{equation}
eos(\rho,h,p,T) = 0 \quad \text{or} \quad \{\rho,h,p,T\} \in \{ eos \}
\end{equation}
We just made the problem harder.

## Parameterizing Constraints with Autoencoders: The Pendulum


<img src="slides/autoencoders.png" width=350 style="float:right;">


Solve for $x(t),y(t)$ stationary on
\begin{equation}
L(x,y) = \frac{1}{2}m\left(\dot{x}^2 + \dot{y}^2\right) - m g y
\end{equation}
subject to
\begin{equation}
x^2 + y^2 = R^2
\end{equation}

$\theta$ is one instance of an **autoencoder**. We can look for one without the geometric intuition.

Solve the minimization problem on data set $x$ for parameters $a$
\begin{equation}
\min_a \sum_x \left( x - D(E(x;a);a) \right)^2
\end{equation}
where $q = E(x)$
with `len(q)<len(x)`.

# Back to Multiphase EOS

Solve for $q_1(t)$ and $q_2(t)$ such that:
\begin{align}
\partial_t \rho(q_1,q_2) & = \nabla \cdot \mathbf{k} \nabla p(q_1,q_2) + r \\
\partial_t \rho h(q_1,q_2)-p & = \nabla \cdot \mathbf{k'}\nabla T(q_1,q_2) + s
\end{align}
where $\rho(q_1,q_2)$ etc. are the back of an autoencoder:
\begin{equation}
\left\{ \begin{array}{c}
T\\ p\\ \rho\\ h
\end{array}\right\} \rightarrow  E \rightarrow 
\left\{ \begin{array}{c} q_1\\q_2 \end{array} \right\}\rightarrow D \rightarrow 
\left\{ \begin{array}{c}
T\\ p\\ \rho\\ h
\end{array}\right\}
\end{equation}


## Two step process to make this simulator:

<img src='figures/autoencoder_balance_detailed.png' width=800>

The autoencoder task defines how part of the program needs to behave.  
**Only training the material representation, not the balance laws.**

## Let's not just throw Neural Networks at it

Softly-Piecewise Polynomials

<img src='figures/classifier_network.png' width=800>

- Use our physical intuition craft a network. 
- We want piecewise smoothness for differentiability.
- Allows for unsupervised learning of phases based on what makes a good fit.
- Three layers deep with an internal polynomial basis up to 6th order.

## State Identification

<center>
<video controls preload="auto" width="500">
<source src="slides/evo.mp4" type="video/mp4">
Your browser does not support the video tag.
</video>
</center>

Rendering the $p-T-\rho$ surface during training, colored by learned class ids.

# Methodology

1. Define tests that validate the program.
1. Describe the material by providing a dataset from experiments or theory.
2. Define a training goal for the computer to optimize.

<img src="slides/user_workflow.png" width=1000>


In [4]:
IFrame('slides/3d_zoo.html',width=1100,height=600)

## Use this to transform the spatial system:

<img src='figures/latent_integration.png' width=800>


## Glue together modules like any simulator:

<img src='figures/autoencoder_fvm.png' width=800>

Some modules can be hand-written, some can be trained.

## We can solve flow problems the grid blocks with fluxes:

<center>
<video controls preload="auto" width="800">
<source src="slides/water_column.mp4" type="video/mp4">
Your browser does not support the video tag.
</video>
</center>

Steam column seperation into liquid and gas.

## We can do arbitrary meshes:

<center>
<video controls preload="auto" width="800">
<source src="slides/2d_curved_reservoir.mp4" type="video/mp4">
Your browser does not support the video tag.
</video>
</center>

Phase seperation in a gas cap reservoir.

**Boundary conditions, meshing, integration is the same as our old simulator!**

## We can do arbitrary meshes:
<center>
<video controls preload="auto" width="800">
<source src="slides/2d_curved_reservoir_q.mp4" type="video/mp4">
Your browser does not support the video tag.
</video>
</center>

Phase seperation in a gas cap reservoir.

**But the unknown fields are different.** (Note they're in $(-0.5,0.5)$)

## Conclusions

Think from a *differential programming* perspective.

- Deep learning can replace and improve hand-baked equations and algorithms
- Can do unsupervised phase classification
- **Only training the material representation, not the balance laws**

**TODO:**

- Pure autoencoder loss function doesn't yield perfectly robust EOSes
- Add in sparsity constraints
- Close loop on testing with reinforcement learning
- Rewrite it all to differentiate through the entire simulator

**Find it here:** https://github.com/afqueiruga/LatentPrimaries  


# Thank you

**Find it here:** https://github.com/afqueiruga/LatentPrimaries  

## Software Engineering

- Training code and simulation written with TensorFlow
- Multiple steps besides just SGD and train/test losses; tensorboard and notebooks don't do the trick
- Completely new framework needs new software to interpret the results
- Build a UI in DASH to evaluate training and testing results

## Testing the Method

Three different extents:
1. Linear Equation of State  
  -  **Test!**
  - Reduces to single phase Darcy's law problem (slide 1)
  - $ p = 10^5+[-10^3, 10^3] Pa,\quad T = [ 19, 21 ] ^o C$
2. Water Liquid-Gas Regime  
  - One phase boundary  
  - $ p = [100,5\times 10^5] Pa, \quad T = [274,594] K$
3. Water Solid-Liquid-Gas-Supercritical Regimes  
  - No linear mapping to latent space  
  - Entire span
  - $ p = [6\times 10^{-6},3\times 10^8]Pa, \quad T = [150,1000] K $
